In [2]:
from env_hiv import *
from gymnasium.wrappers import TransformReward
from ray.rllib.algorithms.ppo import PPOConfig
from gymnasium.wrappers import TimeLimit
from ray.tune.registry import register_env
select_env = "HIV"

env = HIVPatient()
def rew(state,action, env) : 
    return-(
                env.Q * state[4]
                + env.R1 * action[0] ** 2
                + env.R2 * action[1] ** 2
                - env.S * state[5]
            )
low_r, high_r = rew(env.lower, [1,1],env),rew(env.upper, [1,1],env)
env = TransformReward(env, lambda r: (r-low_r)/(high_r-low_r))


register_env(select_env, lambda x: TimeLimit(env,200))
config = (  # 1. Configure the algorithm,
    PPOConfig()
    .environment('HIV')
    .rollouts(num_rollout_workers=18)
    .framework("torch")
    .resources(num_gpus=1)
    .training(model={"fcnet_hiddens": [256, 256]},gamma=0.9, lr=0.01, kl_coeff=0.3)
    .evaluation()
)


algo = config.build()  # 2. build the algorithm,

from tqdm import trange
pbar = trange(1000)
for _ in pbar : 
    r = algo.train()['episode_reward_mean'] * (high_r -low_r) + low_r
    pbar.set_postfix(total_return = "{:e}".format(r))

#algo.evaluate()  # 4. and evaluate it.

/home/tordjx/miniconda3/envs/mujoco_py/lib/python3.8/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/tordjx/miniconda3/envs/mujoco_py/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/tordjx/miniconda3/envs/mujoco_py/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: T

KeyboardInterrupt: 